# Dependencies

In [1]:
import numpy as np
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from scipy import signal
import librosa as lr
import librosa.feature as lrf
from scipy.signal import welch
import pywt
from pywt import *
from scipy.signal import periodogram
#from pyemd import emd
from scipy.signal import hilbert
from scipy.stats import linregress, skew, kurtosis
from scipy.fft import fft, fftfreq

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import nolds
from pyentrp import entropy as ent
from scipy.signal import detrend
from nolds import dfa
import utils, training, constants

In [2]:
dataset = utils.data_loader(path=constants.MAIN_CSV_FILE)
reduced_dataset = utils.channel_selection(dataset, constants.SELECTED_CHANNELS)

subject_idx = {1: [0, 187],
               2: [188, 319],
               3: [320, 470],
               4: [471, 617],
               5: [618, 841],
               6: [842, 1007],
               7: [1008, 1109],
               8: [1110, 1373],
               9: [1374, 1687],
               10: [1688, 1795],
               11: [1796, 2021]} 

In [4]:
from tqdm import tqdm
model_dfs = []
for model in tqdm(constants.ALL_MODELS):
    one_out_results = {}
    for sub in subject_idx:
        one_out = reduced_dataset[reduced_dataset["subjects"] == sub].drop('subjects', axis=1)
        X_train = (reduced_dataset.drop(one_out.index)).drop('subjects', axis=1).drop('label', axis=1)
        y_train = (reduced_dataset.drop(one_out.index)).drop('subjects', axis=1)['label']
        X_test = one_out.drop('label', axis=1)
        y_test = one_out['label']
        data = [X_train, X_test, y_train, y_test]
        one_out_results[sub] = training.model_training(data, model, stats=False, cm=False)
        one_out_results_df = pd.DataFrame.from_dict(one_out_results).T
        one_out_results_df.to_csv("outs/mel_testing/final_mels" + model + "_one_out.csv")
        model_dfs.append(one_out_results_df)

100%|██████████| 8/8 [18:04<00:00, 135.53s/it]
